In [147]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

In [148]:
%%time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from requests import get

from warnings import warn
from time import sleep
from random import randint

import seaborn as sns

import requests


CPU times: user 56 µs, sys: 5 µs, total: 61 µs
Wall time: 70.8 µs


In [29]:
#url_imdb = "https://www.allocine.fr/film/meilleurs/"

In [127]:
#base_url = "https://www.allocine.fr/film/meilleurs/?page=2"

url = "https://www.allocine.fr/films/?page=372"

In [128]:
response = requests.get(url)
bs_allocine = bs(response.text, "html.parser")

In [129]:
films_allocine = bs_allocine.findAll("li", attrs = {'class' : "mdl"})

In [130]:
len(films_allocine)

15

In [131]:
film_allocine = films_allocine[0]
titre_allocine = film_allocine.find('a', class_='meta-title-link').text
titre_allocine

'Un, deux, trois, soleil'

In [137]:
notes_presse = film_allocine.find_all('span', class_='stareval-note')[0]
print(notes_presse.text)

notes_spectateurs = film_allocine.find_all('span', class_='stareval-note')[1]
print (notes_spectateurs.text)



3,1
--


In [140]:
notes_spectateurs.text == "--"

True

### GENERALISATION POUR LES FILMS D'UNE PAGE 

In [134]:
titre_allocine = []
note_allocine_presse = []
note_allocine_spect = []

for film_allocine in films_allocine :
    titre_allocine.append(film_allocine.find('a', class_='meta-title-link').text)
    note_allocine_presse.append(film_allocine.find_all('span', class_='stareval-note')[0].text)
    note_allocine_spect.append(film_allocine.find_all('span', class_='stareval-note')[1].text)


df_allocine = pd.DataFrame(list(zip(titre_allocine, note_allocine_presse, note_allocine_spect)), columns=["Titre", "Note_presse","Note_spectateur"])

In [135]:
df_allocine

,Titre,Note_presse,Note_spectateur
0,"Un, deux, trois, soleil","3,0",--
1,Braquage à l'Anglaise,"3,8","3,5"
2,Die Hard : belle journée pour mourir,"1,7","2,1"
3,Enfance clandestine,"3,6","3,9"
4,Un amour à New York,"3,2","2,9"
5,Noël au château,"3,2",--
6,L'Ultime razzia,"3,4","3,8"
7,Rushmore,"3,6","3,8"
8,Ce soir je vais tuer l'assassin de mon fils,"3,7",--
9,La Dame de fer,"2,3","2,9"


### GENERALISATION POUR LES FILMS DE TOUTES LES PAGES 

In [ ]:
%%time
# Définir la base de l'URL
base_url = "https://www.allocine.fr/films/?page="

titre_allocine = []
note_allocine_presse = []
note_allocine_spect = []

# Boucle sur les pages
for page in range(1, 3000):  #7788 normalement 50 juste pour tester là
    url_bis = f"{base_url}{page}"

    # Envoyer une requête GET à l'URL
    response = requests.get(url_bis)
    
    if response.status_code == 200:
        # Utiliser BeautifulSoup pour analyser le contenu de la page
        bs_allocine = bs(response.text, "html.parser")

        # Trouver tous les films sur la page
        films_allocine = bs_allocine.findAll("li", attrs = {'class' : "mdl"})
        
        # Boucler sur chaque film et extraire les informations
        for film_allocine in films_allocine:
            titre_allocine.append(film_allocine.find('a', class_='meta-title-link').text)
            
            #note_allocine_presse.append(film_allocine.find_all('span', class_='stareval-note')[0].text)
            #note_allocine_spect.append(film_allocine.find_all('span', class_='stareval-note')[1].text)
            note_presse_tag = film_allocine.find_all('span', class_='stareval-note')
            note_allocine_presse.append(note_presse_tag[0].text if note_presse_tag else None)
    
            note_spect_tag = film_allocine.find_all('span', class_='stareval-note')
            note_allocine_spect.append(note_spect_tag[1].text if note_spect_tag and len(note_spect_tag) > 1 else None)
    
    else:
        print(f"Échec de la requête pour la page {page}")

# Créer un DataFrame avec les données collectées
df_allocine = pd.DataFrame(list(zip(titre_allocine, note_allocine_presse, note_allocine_spect)), columns=["Titre", "Note_presse","Note_spectateur"])

# Afficher le DataFrame
#print(df_allocine)


In [156]:
df_allocine

,Titre,Note_presse,Note_spectateur
0,Napoléon,"2,9","2,3"
1,Hunger Games: la Ballade du serpent et de l'oi...,"3,1","3,8"
2,Le Garçon et le Héron,"4,2","3,8"
3,La Tresse,"2,4","4,2"
4,Killers of the Flower Moon,"4,4","4,1"
...,...,...,...
1480,Rocky Balboa,"3,7","3,7"
1481,Troll,"2,3",--
1482,Tous en scène,"3,6","4,0"
1483,American Sniper,"3,8","4,1"


In [143]:
df_allocine.to_csv("rate", index=False)

In [145]:
len(df_allocine["Titre"].unique())

685